In [ ]:
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
import pre_processing as pp
import csv
import pandas as pd
import tensorflow as tf

# Dataset ting

In [ ]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('data/6-gon-correct-procrustes.csv')

df = df.iloc[:, :14]
dataset = df.drop(columns=['target_edge_length'])

training = dataset.sample(frac=0.85)
testing = dataset.drop(training.index)

train_features = training.iloc[:, :12]
train_labels = training.iloc[:, -1:]

test_features = testing.iloc[:, :12]
test_labels = testing.iloc[:, -1:]

# Model ting

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_addons as tfa
import os, datetime

model_path = "model/nn1-6gon-1"
BATCH_SIZE = 512
EPOCHS = 3000
INITIAL_LEARNING_RATE = 1e-2

def model_setup():
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(12,)),
        layers.BatchNormalization(),

        layers.Dense(24, activation='relu'),
        layers.BatchNormalization(),

        layers.Dense(24, activation='relu'),
        layers.BatchNormalization(),

        layers.Dense(1),
    ])
    return model

def train_model(model, model_epochs=EPOCHS, model_batch_size=BATCH_SIZE, learning_rate=INITIAL_LEARNING_RATE):
    # Callbacks
    # logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    # tensorboard = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=EPOCHS//3, min_delta=0.0001)

    # Learning rate scheduler
    decay_steps= 1_000
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        INITIAL_LEARNING_RATE, 
        decay_steps, 
        3e-4
        )

    # Initialize model
    model.compile(loss=tf.losses.MeanAbsoluteError(),
                optimizer=tf.optimizers.Adam(
                learning_rate=lr_schedule,
            ),
            )

    # Train model and write progress to history (for graphing)
    history = model.fit(train_features,
                        train_labels,
                        epochs=model_epochs,
                        batch_size=model_batch_size,
                        validation_split=0.18,
                        verbose=2,
                        callbacks=[checkpoint],
                        )
    return history


# %load_ext tensorboard
# %tensorboard --logdir logs
model = model_setup()
history = train_model(model)

# ======================
#       EVALUATION
# ======================
train_acc = model.evaluate(
    train_features, train_labels, verbose=0)
test_acc = model.evaluate(
    test_features, test_labels, verbose=0)
print('Training data loss: %.3f, Test data loss: %.3f' %
        (train_acc, test_acc))      


In [ ]:
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("MAE")
plt.ylim(0.33, 0.6)

In [ ]:
# ======================
#  EVALUATE SAVED MODEL 
# ======================

saved_model = tf.keras.models.load_model('model/nn1-6gon')

train_acc = saved_model.evaluate(
    train_features, train_labels, verbose=0)
test_acc = saved_model.evaluate(
    test_features, test_labels, verbose=0)
print('Training data loss: %.3f, Test data loss: %.3f' %
        (train_acc, test_acc))

In [ ]:


thesis_model = tf.keras.models.load_model('model/thesis-internal-node-count-6gon')
thesis_model.summary()

train_acc = thesis_model.evaluate(
    train_features, train_labels, verbose=0)
test_acc = thesis_model.evaluate(
    test_features, test_labels, verbose=0)
print('Training data loss: %.3f, Test data loss: %.3f' %
        (train_acc, test_acc))